In [43]:
import os

In [44]:
#os.chdir("../")

In [45]:
#%pwd

In [46]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    scaler_path: Path 
    test_data_path: Path
    model_path: Path
    metric_file_name: Path

In [47]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
from pathlib import Path

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config["root_dir"]])

        return ModelEvaluationConfig(
    root_dir=Path(config["root_dir"]),
        test_data_path=Path(config["test_data_path"]),
        model_path=Path(config["model_path"]),
        metric_file_name=Path(config["metric_file_name"]),
        scaler_path=Path(self.config.model_trainer["scaler_path"])  # ✅ reuse from training config
    )



In [48]:
import joblib
import pandas as pd
import os, json
from sklearn.metrics import accuracy_score, f1_score, classification_report

class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def evaluate(self):
        # Load test data
        df = pd.read_csv(self.config.test_data_path)
        X_test = df.drop("treatment", axis=1)
        y_test = df["treatment"]

        # Load scaler ✅
        scaler = joblib.load(self.config.scaler_path)
        X_test_scaled = scaler.transform(X_test)

        # Load model
        model = joblib.load(self.config.model_path)

        # Predict
        y_pred = model.predict(X_test_scaled)  # ✅ scaled test data

        # Metrics
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, pos_label=1)
        report = classification_report(y_test, y_pred, target_names=["No", "Yes"], output_dict=True)

        # Save metrics
        metrics = {
            "accuracy": acc,
            "f1_score": f1,
            "classification_report": report
        }

        os.makedirs(os.path.dirname(self.config.metric_file_name), exist_ok=True)
        with open(self.config.metric_file_name, "w") as f:
            json.dump(metrics, f, indent=4)

        # Print
        print(f"\n✅ Evaluation Completed:\nAccuracy: {acc:.4f} | F1 Score: {f1:.4f}")
        print("📄 Classification Report:")
        print(classification_report(y_test, y_pred, target_names=["No", "Yes"]))


In [49]:
try:
    config = ConfigurationManager()
    eval_config = config.get_model_evaluation_config()
    evaluator = ModelEvaluation(config=eval_config)
    evaluator.evaluate()
except Exception as e:
    raise e

[2025-07-21 19:02:57,834: INFO: common: yaml file: D:\end to end mental_health_prediction\config.yaml loaded successfully]
[2025-07-21 19:02:57,842: INFO: common: yaml file: D:\end to end mental_health_prediction\params.yaml loaded successfully]
[2025-07-21 19:02:57,847: INFO: common: yaml file: D:\end to end mental_health_prediction\schema.yaml loaded successfully]
[2025-07-21 19:02:57,850: INFO: common: created directory at: artifacts]
[2025-07-21 19:02:57,852: INFO: common: created directory at: artifacts/model_evaluation]

✅ Evaluation Completed:
Accuracy: 0.7410 | F1 Score: 0.7510
📄 Classification Report:
              precision    recall  f1-score   support

          No       0.69      0.77      0.73       114
         Yes       0.79      0.72      0.75       137

    accuracy                           0.74       251
   macro avg       0.74      0.74      0.74       251
weighted avg       0.75      0.74      0.74       251



c:\Users\Sujal Khant\.conda\envs\health\lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
